In [1]:
#importing libraries
import math
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly_express as px
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
netflix_data = pd.read_csv('NetflixDataset.csv',encoding='latin-1', index_col = 'Title')
netflix_data.head(10)

,Genre,Tags,Languages,Country Availability,Runtime,Director,Writer,Actors,View Rating,IMDb Score,...,Awards Nominated For,Boxoffice,Release Date,Netflix Release Date,Production House,Netflix Link,Summary,Series or Movie,IMDb Votes,Image
Title,,,,,,,,,,,,,,,,,,,,,
Lets Fight Ghost,"Crime, Drama, Fantasy, Horror, Romance","Comedy Programmes,Romantic TV Comedies,Horror ...","Swedish, Spanish",Thailand,< 30 minutes,Tomas Alfredson,John Ajvide Lindqvist,"Lina Leandersson, Kåre Hedebrant, Per Ragnar, ...",R,7.9,...,57.0,"$21,22,065",12-Dec-08,04-03-2021,"Canal+, Sandrew Metronome",https://www.netflix.com/watch/81415947,A med student with a supernatural gift tries t...,Series,205926.0,https://occ-0-4708-64.1.nflxso.net/dnm/api/v6/...
HOW TO BUILD A GIRL,Comedy,"Dramas,Comedies,Films Based on Books,British",English,Canada,1-2 hour,Coky Giedroyc,Caitlin Moran,"Cleo, Paddy Considine, Beanie Feldstein, Dónal...",R,5.8,...,NaN,"$70,632",08-May-20,04-03-2021,"Film 4, Monumental Pictures, Lionsgate",https://www.netflix.com/watch/81041267,"When nerdy Johanna moves to London, things get...",Movie,2838.0,https://occ-0-1081-999.1.nflxso.net/dnm/api/v6...
The Con-Heartist,"Comedy, Romance","Romantic Comedies,Comedies,Romantic Films,Thai...",Thai,Thailand,> 2 hrs,Mez Tharatorn,"Pattaranad Bhiboonsawade, Mez Tharatorn, Thods...","Kathaleeya McIntosh, Nadech Kugimiya, Pimchano...",NaN,7.4,...,NaN,NaN,03-Dec-20,03-03-2021,NaN,https://www.netflix.com/watch/81306155,After her ex-boyfriend cons her out of a large...,Movie,131.0,https://occ-0-2188-64.1.nflxso.net/dnm/api/v6/...
Gleboka woda,Drama,"TV Dramas,Polish TV Shows,Social Issue TV Dramas",Polish,Poland,< 30 minutes,NaN,NaN,"Katarzyna Maciag, Piotr Nowak, Marcin Dorocins...",NaN,7.5,...,4.0,NaN,14-Jun-11,03-03-2021,NaN,https://www.netflix.com/watch/81307527,A group of social welfare workers led by their...,Series,47.0,https://occ-0-2508-2706.1.nflxso.net/dnm/api/v...
Only a Mother,Drama,"Social Issue Dramas,Dramas,Movies Based on Boo...",Swedish,"Lithuania,Poland,France,Italy,Spain,Greece,Bel...",1-2 hour,Alf Sjöberg,Ivar Lo-Johansson,"Hugo Björne, Eva Dahlbeck, Ulf Palme, Ragnar F...",NaN,6.7,...,1.0,NaN,31-Oct-49,03-03-2021,NaN,https://www.netflix.com/watch/81382068,An unhappily married farm worker struggling to...,Movie,88.0,https://occ-0-2851-41.1.nflxso.net/dnm/api/v6/...
Snowroller,Comedy,"Sports Movies,Sports Comedies,Comedies,Swedish...","Swedish, English, German, Norwegian","Lithuania,Poland,France,Italy,Spain,Greece,Cze...",1-2 hour,Lasse Åberg,"Lasse Åberg, Bo Jonsson","Lasse Åberg, Cecilia Walton, Eva Millberg, Jon...",NaN,6.6,...,NaN,NaN,04-Oct-85,03-03-2021,NaN,https://www.netflix.com/watch/81382187,"Two friends take a ski trip to the Alps, where...",Movie,5926.0,https://occ-0-2851-41.1.nflxso.net/dnm/api/v6/...
The Invisible,"Crime, Drama, Fantasy, Mystery, Thriller","Thriller Movies,Movies Based on Books,Supernat...",English,"Lithuania,Poland,France,Italy,Spain,Greece,Cze...",1-2 hour,David S. Goyer,"Mats Wahl, Mick Davis, Christine Roum","Marcia Gay Harden, Margarita Levieva, Chris Ma...",PG-13,6.2,...,1.0,"$2,05,78,909",27-Apr-07,03-03-2021,"Touchstone Pictures, Spyglass Entertainment",https://www.netflix.com/watch/81382078,"Critically injured after a brutal beating, a t...",Movie,34738.0,https://occ-0-2851-41.1.nflxso.net/dnm/api/v6/...
The Simple Minded Murderer,Drama,"Social Issue Dramas,Dramas,Movies Based on Boo...","Scanian, Swedish","Lithuania,Poland,France,Italy,Spain,Greece,Cze...",1-2 hour,Hans Alfredson,Hans Alfredson,"Maria Johansson, Hans Alfredson, Stellan Skars...",NaN,7.6,...,2.0,NaN,01-Sep-85,03-03-2021,"Svensk Filmindustri, Svenska Filminstitutet",https://www.netflix.com/watch/81382075,"A good-natured farmhand, perpetually terrorize...",Movie,2870.0,https://occ-0-2851-41.1.nflxso.net/dnm/api/v6/...
To Kill a Child,"Short, Drama","Dramas,Swedish Movies",Spanish,"Lithuania,Poland,France,Italy,Spain,Greece,Cze...",< 30 minutes,"José Esteban Alenda, César Esteban A

In [3]:
netflix_data.index = netflix_data.index.str.title()

In [4]:
netflix_data.rename(columns={'View Rating':'ViewerRating'}, inplace=True)

In [5]:
Language = netflix_data.Languages.str.get_dummies(',')
Lang = Language.columns.str.strip().values.tolist()
Language = netflix_data['Languages']
Language_Count = dict()
for i in Lang:
    p = Language.str.count(i).sum()
    Language_Count[i] = int(p)
print(len(Language_Count))


184


In [6]:
Languagenetflix_data.Languages.str.get_dummies(',')

NameError: name 'Languagenetflix_data' is not defined

In [ ]:
Language_Count

In [ ]:
Language.str.count('Amharic').sum()

In [ ]:
Language_Count = {k: v for k, v in sorted(Language_Count.items(), key=lambda item: item[1], reverse = True)}
top_languages = {"Languages": list(Language_Count.keys()), "Count": list(Language_Count.values())}

In [ ]:
Language_Count


In [ ]:
top_languages['Languages']

In [ ]:
new=pd.DataFrame(top_languages)[:10]

In [ ]:
new

In [ ]:
plt.figure(figsize = (15,5))
sns.barplot(data = new, y = new.Count, x = "Languages")
plt.show()

In [ ]:
fig = px.bar(pd.DataFrame(top_languages)[:10], y = 'Languages', x = 'Count', orientation = 'h', title = 'Most Available Languages', color = 'Count', color_continuous_scale = px.colors.qualitative.Prism).update_yaxes(categoryorder = 'total ascending')
fig.show()

In [ ]:
Genres = netflix_data.Genre.str.get_dummies(',')
Genre = Genres.columns.str.strip().values.tolist()
Genres = netflix_data['Genre']
Genre_Count = dict()
for i in Genre:
    p = Genres.str.count(i).sum()
    Genre_Count[i] = int(p)
print(len(Genre_Count))

In [ ]:
Genre_Count = {k: v for k, v in sorted(Genre_Count.items(), key=lambda item: item[1], reverse = True)}
top_genres = {"Genre": list(Genre_Count.keys()), "Count": list(Genre_Count.values())}

In [ ]:
fig = px.bar(pd.DataFrame(top_genres)[:10], y = 'Genre', x = 'Count', orientation = 'h', title = 'Genres with maximum content', color = 'Count', color_continuous_scale = px.colors.qualitative.Prism).update_yaxes(categoryorder = 'total ascending')
fig.show()

In [ ]:
top_15 = netflix_data.sort_values(by = ['IMDb Score'], ascending = False).head(15)
plt.figure(figsize = (15,5))
sns.barplot(data = top_15, y = top_15.index, x = "IMDb Score")
plt.show()

In [ ]:
netflix_data = netflix_data[~netflix_data.index.duplicated()]

In [ ]:
netflix_data.index.duplicated().sum()

In [ ]:
netflix_data.index.isnull().sum()

In [ ]:
netflix_data['Genre'] = netflix_data['Genre'].astype('str')
print((netflix_data['Genre'] == 'nan').sum())

In [ ]:
netflix_data['Tags'] = netflix_data['Tags'].astype('str')
print((netflix_data['Tags'] == 'nan').sum())

In [ ]:
print(((netflix_data['Genre'] == 'nan') & (netflix_data['Tags'] == 'nan')).sum())
#so these two features can used to recommend movies as no movie can be left unrecommended

In [ ]:
print(netflix_data[['IMDb Score']].describe())
netflix_data['IMDb Score'].mode()
#this feature will be used to sort the movie or series list to represent the recommended items

In [ ]:
netflix_data['IMDb Score'] = netflix_data['IMDb Score'].apply(lambda x: 6.6 if x == 0 or math.isnan(x) else x)
print(netflix_data[['IMDb Score']].describe())
#since no value has suffered for change greater than 0.0003 after replacing the null values with mode value, so we replace the null values with 6.6

In [ ]:
netflix_data['Actors'] = netflix_data['Actors'].astype('str')
netflix_data['ViewerRating'] = netflix_data['ViewerRating'].astype('str')

In [ ]:
def prepare_data(x):
        return str.lower(x.replace(" ", ""))

In [ ]:
new_features = ['Genre', 'Tags', 'Actors', 'ViewerRating']
selected_data = netflix_data[new_features]

In [ ]:
for new_feature in new_features:
    selected_data.loc[:, new_feature] = selected_data.loc[:, new_feature].apply(prepare_data)
selected_data.index = selected_data.index.str.lower()
selected_data.index = selected_data.index.str.replace(" ",'')
selected_data.head(2)

In [ ]:
def create_soup(x):
    return x['Genre'] + ' ' + x['Tags'] + ' ' +x['Actors']+' '+ x['ViewerRating']

In [ ]:
selected_data.loc[:, 'soup'] = selected_data.apply(create_soup, axis = 1)
selected_data.head(2)

In [ ]:
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(selected_data['soup'])
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [ ]:
selected_data.reset_index(inplace = True)
selected_data.head(2)

In [ ]:
indices = pd.Series(selected_data.index, index=selected_data['Title'])
indices

In [ ]:
result = 0
def get_recommendations(title, cosine_sim):
    global result
    title=title.replace(' ','').lower()
    idx = indices[title]
    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores of the 50 most similar movies
    sim_scores = sim_scores[1:51]
    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]
    # Return the top 10 most similar movies
    result =  netflix_data.iloc[movie_indices]
    result.reset_index(inplace = True)
    return result

In [ ]:
df = pd.DataFrame()
movienames = ['Annabelle Comes Home','The Nun', 'Insidious: The Last Key', 'Conjuring 2', 'Insidious: Chapter 3']
languages = ['English', 'Hindi']
for moviename in movienames:
    get_recommendations(moviename,cosine_sim2)
    for language in languages:
        df = pd.concat([result[result['Languages'].str.count(language) > 0], df], ignore_index=True)
df.drop_duplicates(keep = 'first', inplace = True)
df.sort_values(by = 'IMDb Score', ascending = False, inplace = True)

In [ ]:
print(df.shape)
print(df.head())